<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática 3.0**
*by [sigmoidal.ai](https://sigmoidal.ai)*

---

# Dados de treino e teste

Parte vital de qualquer projeto de Ciência de Dados é fazer o split dos dados da forma e no momento certo.

<center><img src="https://www.machinecurve.com/wp-content/uploads/2020/11/feed-3.jpg" width=400px ></center>


Isso acontece para que não haja vazamento de dados, que pode causar *overfitting*, o que significa, em termos simples, que o nosso modelo não aprendeu a resolver o problema, e sim, decorou as respostas.

<center><img src="https://miro.medium.com/max/1396/1*lARssDbZVTvk4S-Dk1g-eA.png" width=400px ></center>

*Overfitting* é um grande problema em qualquer tipo de trabalho com Machine Learning, como podemos ver na imagem. Um bom modelo não precisa ser perfeito, com uma acurácia de 100%. Inclusive, uma acurácia de 100% seria um fortíssimo indicativo de *overfitting*, sendo motivo de preoucupação em um projeto, e não de celebração.

Bons modelos são generalistas o suficiente para entender as nuances do poroblema e se adaptarem a novos dados de forma efetiva. Isso é o que buscamos. 

Mas como alcançar isso?

É claro que precisamos nos atentar a diversos detalhes e passos no processo de Machine Learning, mas, por enquanto, vamos nos ater ao *split*, ou divisão do conjunto de dados.






## Separando o Dataset

Ao recebermos um conjunto de dados completo, o primeiro passo é serpará-lo, como demonstra a primeira figura. Entãom, dividiremos o conjunto inteiro entre `Treino` e `Teste`, e então separaremos o de `Treino` entre `Treino` e `Validação`.

<center><img src="https://miro.medium.com/max/1400/1*RJS8yV5mBDqrRu7THooH-w.png" width=400px ></center>

### Treino

O conjunto de treino será o maior, e será usado para a análise de dados, para criação do pipeline de processamento desses dados, e também para criar nossa *baseline*, que usaremos para selecionar o modelo a ser otimizado.

### Validação

Esse pedaço do conjunto de treino, será mais ou menos do mesmo tamanho do conjunto de teste, e será usado como um conjunto de teste. Isso significa que o modelo criado não pode ter visto esses dados na análise, assim como os de teste. Por isso esses conjuntos são comumente chamados de *"holdout sets"*, pois vamos "segurá-los" até o momento certo de usá-los.

No caso do conjunto de validação, nós vamos ter construído e selecionado o modelo, ou modelos, a serem otimizados. Nessa parte, usaremos o conjunto para fazer o tuning de parâmetros, e selecionar o melhor modelo, que será usado em produção, entregue ao cliente, etc.

### Teste

Esse conjunto é utilizado apenas no final, com o seu modelo pronto, e otimizado, para testar o resultado do modelo pronto com dados novos, da mesma forma que ele receberá esses dados no mundo real. Aqui é onde saberemos a verdadeira performance do modelo, e avaliaremos a mesma baseado nas métricas de interesse para o problema.

Parece confuso, mas na prática, é mais simples.


In [ ]:
# importar os pacotes necessários
import pandas as pd

# importar o arquivo
df = pd.read_csv("http://dl.dropboxusercontent.com/s/gc1rm540g4i670n/precificacao_housing_plus.csv?dl=0")

# ver as primeiras entradas
print(df.shape)
df.head()

(1460, 35)


,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,8450,7,5,2003,2003,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2,548,0,61,0,0,0,0,0,2,2008,208500
1,2,20,9600,6,8,1976,1976,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,2,460,298,0,0,0,0,0,0,5,2007,181500
2,3,60,11250,7,5,2001,2002,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2,608,0,42,0,0,0,0,0,9,2008,223500
3,4,70,9550,7,5,1915,1970,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,3,642,0,35,272,0,0,0,0,2,2006,140000
4,5,60,14260,8,5,2000,2000,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,3,836,192,84,0,0,0,0,0,12,2008,250000


Ao carregar seu conjunto de dados, separe uma fração dele para ser seu conjunto de teste. É normal ver a proporção 70/15/15 para os conjuntos, mas não é uma regra.

Ainda mais hoje em dia com Big Data, não é absurdo uma proporção 95/2.5/2.5, dado o grande volume de dados, e a importância de ser ter um grande conjunto de treino.

No exemplo abaixo, vou usar 15% do dataset como conjunto de teste.

In [ ]:
# criando o conjunto de teste
test = df.sample(frac=0.15, random_state=0)

# verificando o conjunto
print(test.shape)
test.head()

(219, 35)


,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
529,530,20,32668,6,3,1957,1975,1219,0,816,2035,2515,0,0,2515,1,0,3,0,4,2,9,2,2,484,0,0,200,0,0,0,0,3,2007,200624
491,492,50,9490,6,7,1941,1950,403,165,238,806,958,620,0,1578,1,0,1,0,3,1,5,2,1,240,0,0,32,0,0,0,0,8,2006,133000
459,460,50,7015,5,4,1950,1950,185,0,524,709,979,224,0,1203,1,0,1,0,3,1,5,1,1,352,0,0,248,0,0,0,0,7,2009,110000
279,280,60,10005,7,5,1977,1977,392,0,768,1160,1156,866,0,2022,0,0,2,1,4,1,8,1,2,505,288,117,0,0,0,0,0,3,2008,192000
655,656,160,1680,6,5,1971,1971,0,0,525,525,525,567,0,1092,0,0,1,1,3,1,6,0,1,264,0,0,0,0,0,0,0,3,2010,88000


Uma vez selecionado nosso teste, precisamos retirar as entradas selecionadas do dataset original, para evitar duplicidade de entradas e vazamento de dados.

In [ ]:
# drop das linhas de teste
df = df.drop(test.index)

# verificando o shape do df
df.shape

(1241, 35)

In [ ]:
# resetando o index dos conjuntos
df.reset_index()
test.reset_index()

,index,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,529,530,20,32668,6,3,1957,1975,1219,0,816,2035,2515,0,0,2515,1,0,3,0,4,2,9,2,2,484,0,0,200,0,0,0,0,3,2007,200624
1,491,492,50,9490,6,7,1941,1950,403,165,238,806,958,620,0,1578,1,0,1,0,3,1,5,2,1,240,0,0,32,0,0,0,0,8,2006,133000
2,459,460,50,7015,5,4,1950,1950,185,0,524,709,979,224,0,1203,1,0,1,0,3,1,5,1,1,352,0,0,248,0,0,0,0,7,2009,110000
3,279,280,60,10005,7,5,1977,1977,392,0,768,1160,1156,866,0,2022,0,0,2,1,4,1,8,1,2,505,288,117,0,0,0,0,0,3,2008,192000
4,655,656,160,1680,6,5,1971,1971,0,0,525,525,525,567,0,1092,0,0,1,1,3,1,6,0,1,264,0,0,0,0,0,0,0,3,2010,88000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,1452,1453,180,3675,5,5,2005,2005,547,0,0,547,1072,0,0,1072,1,0,1,0,2,1,5,0,2,525,0,28,0,0,0,0,0,5,2006,145000
142,113,114,20,21000,6,5,1953,1953,35,869,905,1809,2259,0,0,2259,1,0,2,0,3,1,7,2,2,450,166,120,192,0,0,0,0,10,2007,217000
143,1282,1283,20,8800,5,7,1977,2008,532,144,364,1040,1040,0,0,1040,0,0,2,0,3,1,5,0,2,484,0,0,0,0,288,0,0,9,2009,150500
144,1163,1164,90,12900,4,4,1969,1969,1198,0,0,1198,1258,0,0,1258,2,0,0,2,0,2,6,0,2,400,120,0,0,0,0,0,0,1,2008,108959


Com nosso conjunto de testes reservado, podemos fazer o split no conjunto de treino para o conjunto de validação com o Scikit-Learn.

In [ ]:
# importando o módulo necessário
from sklearn.model_selection import train_test_split

In [ ]:
# drop do Target
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [ ]:
# fazendo o split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(df.shape)

(1054, 34)
(187, 34)
(1054,)
(187,)
(1241, 35)
